<a href="https://colab.research.google.com/github/hbabar1215/nhanes_inferential_2021_23/blob/main/nhanes_inferential_2021_23_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install R packages needed
install.packages("ggplot2")
install.packages("lmtest")
install.packages("readr")
install.packages("haven")
install.packages("dplyr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘zoo’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
# Load the packages
library(ggplot2)
library(lmtest)
library(readr)
library(haven)
library(dplyr)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
# Question 1: "Is there an association between marital status (married or not married) and education level (bachelor’s degree or higher vs. less than a bachelor’s degree)?"

In [4]:
# Load Demographic data
df_demo <- read_xpt("/content/DEMO_L.xpt")
head(df_demo)

SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,⋯,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVSTRA,SDMVPSU,INDFMPIR
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
130378,12,2,1,43,NA,5,6,2,NA,⋯,NA,NA,NA,NA,NA,50055.45,54374.46,173,2,5.00
130379,12,2,1,66,NA,3,3,2,NA,⋯,NA,NA,NA,NA,NA,29087.45,34084.72,173,2,5.00
130380,12,2,2,44,NA,2,2,1,NA,⋯,NA,NA,NA,NA,NA,80062.67,81196.28,174,1,1.41
130381,12,2,2,5,NA,5,7,1,71,⋯,2,2,2,3,NA,38807.27,55698.61,182,2,1.53
130382,12,2,1,2,NA,3,3,2,34,⋯,2,2,3,1,2,30607.52,36434.15,182,2,3.60
130383,12,1,2,3,NA,2,2,NA,NA,⋯,2,2,NA,1,NA,43456.14,0.00,176,2,NA


In [5]:
# Keep columns DMDMARTZ and DMDEDUC2
df_demo_new<- df_demo[, c("DMDMARTZ", "SEQN", "DMDEDUC2")]
head(df_demo_new)

DMDMARTZ,SEQN,DMDEDUC2
<dbl>,<dbl>,<dbl>
1,130378,5
1,130379,5
1,130380,3
NA,130381,NA
NA,130382,NA
NA,130383,NA


In [6]:
# Drop missing data or invalid codes
df_clean <- na.omit(df_demo_new)
head(df_clean)

DMDMARTZ,SEQN,DMDEDUC2
<dbl>,<dbl>,<dbl>
1,130378,5
1,130379,5
1,130380,3
3,130384,2
1,130385,3
1,130386,4


In [7]:
# Recode education level
df_clean <-df_clean %>%
  mutate(
    education_level = ifelse(DMDEDUC2 == 5, "Bachelor's or Higher", "Less than Bachelor's")
  )
  head(df_clean)

DMDMARTZ,SEQN,DMDEDUC2,education_level
<dbl>,<dbl>,<dbl>,<chr>
1,130378,5,Bachelor's or Higher
1,130379,5,Bachelor's or Higher
1,130380,3,Less than Bachelor's
3,130384,2,Less than Bachelor's
1,130385,3,Less than Bachelor's
1,130386,4,Less than Bachelor's


In [8]:
# Recode marital status
df_clean <-df_clean %>%
  mutate(
    marital_status = ifelse(DMDMARTZ== 1, "Married", "Not Married")
  )
  head(df_clean)

DMDMARTZ,SEQN,DMDEDUC2,education_level,marital_status
<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,130378,5,Bachelor's or Higher,Married
1,130379,5,Bachelor's or Higher,Married
1,130380,3,Less than Bachelor's,Married
3,130384,2,Less than Bachelor's,Not Married
1,130385,3,Less than Bachelor's,Married
1,130386,4,Less than Bachelor's,Married


In [9]:
colnames(df_demo_new)

[1] "DMDMARTZ" "SEQN"     "DMDEDUC2"

In [10]:
  # Keep only the new recoded columns
  df_demo_clean <- df_clean %>%
  select(SEQN, marital_status, education_level) %>%
  filter(!is.na(marital_status) & !is.na(education_level))

  head(df_demo_clean)


SEQN,marital_status,education_level
<dbl>,<chr>,<chr>
130378,Married,Bachelor's or Higher
130379,Married,Bachelor's or Higher
130380,Married,Less than Bachelor's
130384,Not Married,Less than Bachelor's
130385,Married,Less than Bachelor's
130386,Married,Less than Bachelor's


In [11]:
# Create table
table_question1 <- table(df_clean$marital_status, df_clean$education_level)
print(table_question1)

             
              Bachelor's or Higher Less than Bachelor's
  Married                     1631                 2505
  Not Married                  994                 2662


In [12]:
# Create contigency table
chi_result <- chisq.test(table_question1)
chi_result


	Pearson's Chi-squared test with Yates' continuity correction

data:  table_question1
X-squared = 129.73, df = 1, p-value < 2.2e-16


In [13]:
cat("Chi-square test p-value:", chi_result$p.value, "\n")

Chi-square test p-value: 4.701728e-30 


In [14]:
# Pritn result
if (chi_result$p.value < 0.05) {
  cat("There is a significant relationship between marital status and education level.\n")
} else {
  cat("There is no significant relationship between marital status and education level.\n")
}

There is a significant relationship between marital status and education level.


Answer: This suggests that there is an association between marital level and education level. People who are married are more likely to have a bachelor's degree or higher.

In [15]:
# Question 2: Is there a difference in the mean sedentary behavior time between those who are married and those who are not married?

In [16]:
# Load PAQ data
df_paq <- read_xpt("/content/PAQ_L.xpt")
df_paq

SEQN,PAD790Q,PAD790U,PAD800,PAD810Q,PAD810U,PAD820,PAD680
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
130378,3,W,45,3,W,45,360
130379,4,W,45,3,W,45,480
130380,1,W,20,0,,NA,240
130384,0,,NA,0,,NA,60
130385,1,D,90,1,W,60,180
130386,1,W,30,1,M,30,180
130387,0,,NA,0,,NA,1200
130388,5,W,180,0,,NA,360
130389,3,W,45,2,M,30,720


In [17]:
# Merge the demographic data and PAQ data
df_merged2 <- merge(df_demo_clean, df_paq, by = "SEQN")
df_merged2

SEQN,marital_status,education_level,PAD790Q,PAD790U,PAD800,PAD810Q,PAD810U,PAD820,PAD680
<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
130378,Married,Bachelor's or Higher,3,W,45,3,W,45,360
130379,Married,Bachelor's or Higher,4,W,45,3,W,45,480
130380,Married,Less than Bachelor's,1,W,20,0,,NA,240
130384,Not Married,Less than Bachelor's,0,,NA,0,,NA,60
130385,Married,Less than Bachelor's,1,D,90,1,W,60,180
130386,Married,Less than Bachelor's,1,W,30,1,M,30,180
130387,Not Married,Bachelor's or Higher,0,,NA,0,,NA,1200
130388,Married,Less than Bachelor's,5,W,180,0,,NA,360
130389,Married,Bachelor's or Higher,3,W,45,2,M,30,720


In [18]:
# Keep only SEQN, marital_status, and PAD680 columns
df_question2 <- df_merged2[, c("SEQN", "marital_status", "PAD680")]
df_question2

SEQN,marital_status,PAD680
<dbl>,<chr>,<dbl>
130378,Married,360
130379,Married,480
130380,Married,240
130384,Not Married,60
130385,Married,180
130386,Married,180
130387,Not Married,1200
130388,Married,360
130389,Married,720


In [19]:
# Remove rows where PAD680 is 7777, 9999, or NA
df_question2 <- df_question2[!(df_question2$PAD680 %in% c(7777, 9999)) & !is.na(df_question2$PAD680), ]
df_question2

,SEQN,marital_status,PAD680
,<dbl>,<chr>,<dbl>
1,130378,Married,360
2,130379,Married,480
3,130380,Married,240
4,130384,Not Married,60
5,130385,Married,180
6,130386,Married,180
7,130387,Not Married,1200
8,130388,Married,360
9,130389,Married,720


In [20]:
# Calculate mean PAD680 by marital status
married_data <- subset(df_question2, marital_status == "Married")$PAD680
not_married_data <- subset(df_question2, marital_status == "Not Married")$PAD680

In [21]:
# Complete t-test
t_test_result <- t.test(married_data, not_married_data)
print(t_test_result)


	Welch Two Sample t-test

data:  married_data and not_married_data
t = -3.8458, df = 7414, p-value = 0.0001212
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -28.121958  -9.132616
sample estimates:
mean of x mean of y 
 353.2871  371.9144 



The mean sedentary behavior time between those who are married is lower (353.29) compared to those who are not married (371.91). These results are significant (t= -3.846, p< 0.05) which means that married individuals spend less time sitting at school, at home, getting to and from places, or with friends including time spent sitting at a desk, traveling in a car or bus, reading, playing cards, watching television, or using a computer.

In [22]:
# Question 3: "How do age and marital status affect systolic blood pressure?"

In [23]:
# Load Blood Pressure - Oscillometric Measurements data
df_bp <- read_xpt("/content/BPXO_L.xpt")
head(df_bp)

SEQN,BPAOARM,BPAOCSZ,BPXOSY1,BPXODI1,BPXOSY2,BPXODI2,BPXOSY3,BPXODI3,BPXOPLS1,BPXOPLS2,BPXOPLS3
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
130378,R,4,135,98,131,96,132,94,82,79,82
130379,R,4,121,84,117,76,113,76,72,71,73
130380,R,4,111,79,112,80,104,76,84,83,77
130386,R,4,110,72,120,74,115,75,59,64,64
130387,R,4,143,76,136,74,145,78,80,80,77
130388,R,5,130,95,128,100,129,104,76,84,86


In [24]:
# Merge demographic and BP data
df_merged3<- merge(df_demo, df_bp, by="SEQN", all= TRUE)
head(df_merged3)

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,⋯,BPAOCSZ,BPXOSY1,BPXODI1,BPXOSY2,BPXODI2,BPXOSY3,BPXODI3,BPXOPLS1,BPXOPLS2,BPXOPLS3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,130378,12,2,1,43,NA,5,6,2,NA,⋯,4,135,98,131,96,132,94,82,79,82
2,130379,12,2,1,66,NA,3,3,2,NA,⋯,4,121,84,117,76,113,76,72,71,73
3,130380,12,2,2,44,NA,2,2,1,NA,⋯,4,111,79,112,80,104,76,84,83,77
4,130381,12,2,2,5,NA,5,7,1,71,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,130382,12,2,1,2,NA,3,3,2,34,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,130383,12,1,2,3,NA,2,2,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [25]:
names(df_merged3)

[1] "SEQN"     "SDDSRVYR" "RIDSTATR" "RIAGENDR" "RIDAGEYR" "RIDAGEMN"
 [7] "RIDRETH1" "RIDRETH3" "RIDEXMON" "RIDEXAGM" "DMQMILIZ" "DMDBORN4"
[13] "DMDYRUSR" "DMDEDUC2" "DMDMARTZ" "RIDEXPRG" "DMDHHSIZ" "DMDHRGND"
[19] "DMDHRAGZ" "DMDHREDZ" "DMDHRMAZ" "DMDHSEDZ" "WTINT2YR" "WTMEC2YR"
[25] "SDMVSTRA" "SDMVPSU"  "INDFMPIR" "BPAOARM"  "BPAOCSZ"  "BPXOSY1" 
[31] "BPXODI1"  "BPXOSY2"  "BPXODI2"  "BPXOSY3"  "BPXODI3"  "BPXOPLS1"
[37] "BPXOPLS2" "BPXOPLS3"

In [26]:
# Recode marital status
df_question3 <-df_merged3 %>%
  mutate(
    marital_status = ifelse(DMDMARTZ== 1, "Married", "Not Married")
  )
  head(df_question3)


,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,⋯,BPXOSY1,BPXODI1,BPXOSY2,BPXODI2,BPXOSY3,BPXODI3,BPXOPLS1,BPXOPLS2,BPXOPLS3,marital_status
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,130378,12,2,1,43,NA,5,6,2,NA,⋯,135,98,131,96,132,94,82,79,82,Married
2,130379,12,2,1,66,NA,3,3,2,NA,⋯,121,84,117,76,113,76,72,71,73,Married
3,130380,12,2,2,44,NA,2,2,1,NA,⋯,111,79,112,80,104,76,84,83,77,Married
4,130381,12,2,2,5,NA,5,7,1,71,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,130382,12,2,1,2,NA,3,3,2,34,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,130383,12,1,2,3,NA,2,2,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [27]:
# Create new table with only the columns that are needed
df_small3 <- df_question3[, c("RIDAGEYR", "marital_status", "BPXOSY3")]
head(df_small3)

,RIDAGEYR,marital_status,BPXOSY3
,<dbl>,<chr>,<dbl>
1,43,Married,132
2,66,Married,113
3,44,Married,104
4,5,NA,NA
5,2,NA,NA
6,3,NA,NA


In [28]:
# Drop missing data or invalid codes
df_clean3 <- na.omit(df_small3)
head(df_clean3)

,RIDAGEYR,marital_status,BPXOSY3
,<dbl>,<chr>,<dbl>
1,43,Married,132
2,66,Married,113
3,44,Married,104
9,34,Married,115
10,68,Not Married,145
11,27,Married,129


In [29]:
# Count the number of married people
table(df_clean3$marital_status)


    Married Not Married 
       3186        2656 

In [30]:
# BP difference between married and not married people
df_clean3 %>%
  group_by(marital_status) %>%
  summarise(average_BP = mean(BPXOSY3, na.rm = TRUE))

marital_status,average_BP
<chr>,<dbl>
Married,122.6099
Not Married,122.8532


In [31]:
# Create two groups for t-test
married_bp <- subset(df_clean3, marital_status == "Married")$BPXOSY3
not_married_bp <- subset(df_clean3, marital_status == "Not Married")$BPXOSY3

In [32]:
# Complete t-test
t_test_result <- t.test(married_bp, not_married_bp, var.equal = FALSE)
print(t_test_result)


	Welch Two Sample t-test

data:  married_bp and not_married_bp
t = -0.50224, df = 5568.8, p-value = 0.6155
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.1930016  0.7063875
sample estimates:
mean of x mean of y 
 122.6099  122.8532 



The married individuals had a mean systolic blood pressure of 122.61 while unmarried had a mean systolic blood pressure of 122.85. The results indicated no statistically significant difference between the two groups,
(t = –0.50, p = 0.616). This shows us that marital status does not effect systolic blood pressure.

In [33]:
# Complete correlation test between age and BP
cor_test <- cor.test(df_clean3$RIDAGEYR, df_clean3$BPXOSY3, use = "complete.obs")
cor_test


	Pearson's product-moment correlation

data:  df_clean3$RIDAGEYR and df_clean3$BPXOSY3
t = 29.969, df = 5840, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.3426610 0.3871162
sample estimates:
      cor 
0.3650967 


A Pearson correlation showed a positive correlation between age and systolic blood pressure (r = 0.365, p < 0.001). This tells us that as age increases, systolic blood pressure increases.

In [34]:
# Question 4: Is there a correlation between self-reported weight and minutes of sedentary behavior?

In [35]:
# Load WHQ data for self-reported weight data WHD020
df_whq <-read_xpt("/content/WHQ_L.xpt")
df_whq

SEQN,WHD010,WHD020,WHD050,WHQ070
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
130378,71,190,200,1
130379,70,220,220,2
130380,60,150,165,1
130384,68,204,212,1
130385,70,240,240,2
130386,68,200,180,2
130387,67,215,215,2
130388,66,270,300,1
130389,67,175,185,1


In [36]:
# Combine PAQ and WHQ data
df_merged4 <-merge(df_paq, df_whq, by = "SEQN", all = TRUE)
df_merged4

SEQN,PAD790Q,PAD790U,PAD800,PAD810Q,PAD810U,PAD820,PAD680,WHD010,WHD020,WHD050,WHQ070
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
130378,3,W,45,3,W,45,360,71,190,200,1
130379,4,W,45,3,W,45,480,70,220,220,2
130380,1,W,20,0,,NA,240,60,150,165,1
130384,0,,NA,0,,NA,60,68,204,212,1
130385,1,D,90,1,W,60,180,70,240,240,2
130386,1,W,30,1,M,30,180,68,200,180,2
130387,0,,NA,0,,NA,1200,67,215,215,2
130388,5,W,180,0,,NA,360,66,270,300,1
130389,3,W,45,2,M,30,720,67,175,185,1


In [37]:
# Keep columns PAD680 and WHD020
df_question4 <- df_merged4[, c("SEQN", "PAD680", "WHD020")]
df_question4

SEQN,PAD680,WHD020
<dbl>,<dbl>,<dbl>
130378,360,190
130379,480,220
130380,240,150
130384,60,204
130385,180,240
130386,180,200
130387,1200,215
130388,360,270
130389,720,175


In [38]:
# Replace invalid values (7777 and 9999) with NaN
df_question4$WHD020[df_question4$WHD020 %in% c(7777, 9999)] <- NA
df_question4$PAD680[df_question4$PAD680 %in% c(7777, 9999)] <- NA

In [39]:
# Drop missing data or invalid codes
df_clean4<- na.omit(df_question4)
head(df_clean4)

,SEQN,PAD680,WHD020
,<dbl>,<dbl>,<dbl>
1,130378,360,190
2,130379,480,220
3,130380,240,150
4,130384,60,204
5,130385,180,240
6,130386,180,200


In [40]:
# Complete correlation test
cor_test <- cor.test(df_clean4$PAD680, df_clean4$WHD020, use = "complete.obs")
cor_test


	Pearson's product-moment correlation

data:  df_clean4$PAD680 and df_clean4$WHD020
t = 14.081, df = 7952, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.1344554 0.1773405
sample estimates:
      cor 
0.1559715 


There is a small but statistically significant correlation between self-reported weight and minutes of sedentary behavior as the Pearson correlation coefficient is 0.1560.

In [41]:
# Question 5: Do married women report greater feelings of tiredness or having little energy than unmarried women?

In [42]:
# Load DPQ data
df_dpq <- read_xpt("/content/DPQ_L.xpt")
df_dpq

SEQN,DPQ010,DPQ020,DPQ030,DPQ040,DPQ050,DPQ060,DPQ070,DPQ080,DPQ090,DPQ100
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
130378,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
130379,0,0,1,0,0,0,0,0,0,0
130380,0,0,1,1,0,0,0,0,0,0
130386,0,0,0,0,0,1,0,0,0,0
130387,0,0,0,0,0,0,0,0,0,NA
130388,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
130389,0,0,0,0,0,0,0,0,0,NA
130390,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
130391,3,3,3,3,3,3,3,2,1,2


In [43]:
# Merge DPQ and demo data
df_merged5 <-merge(df_demo, df_dpq, by = "SEQN", all = FALSE)
df_merged5

SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,⋯,DPQ010,DPQ020,DPQ030,DPQ040,DPQ050,DPQ060,DPQ070,DPQ080,DPQ090,DPQ100
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
130378,12,2,1,43,NA,5,6,2,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
130379,12,2,1,66,NA,3,3,2,NA,⋯,0,0,1,0,0,0,0,0,0,0
130380,12,2,2,44,NA,2,2,1,NA,⋯,0,0,1,1,0,0,0,0,0,0
130386,12,2,1,34,NA,1,1,1,NA,⋯,0,0,0,0,0,1,0,0,0,0
130387,12,2,2,68,NA,3,3,2,NA,⋯,0,0,0,0,0,0,0,0,0,NA
130388,12,2,2,27,NA,4,4,1,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
130389,12,2,1,59,NA,3,3,1,NA,⋯,0,0,0,0,0,0,0,0,0,NA
130390,12,2,2,31,NA,3,3,2,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
130391,12,2,2,33,NA,3,3,2,NA,⋯,3,3,3,3,3,3,3,2,1,2


In [44]:
# Keep columns RIAGENDR, DPQ040, DMDMARTZ
df_question5 <- df_merged5[, c("SEQN", "RIAGENDR", "DPQ040", "DMDMARTZ")]
df_question5

SEQN,RIAGENDR,DPQ040,DMDMARTZ
<dbl>,<dbl>,<dbl>,<dbl>
130378,1,NA,1
130379,1,0,1
130380,2,1,1
130386,1,0,1
130387,2,0,3
130388,2,NA,1
130389,1,0,1
130390,2,NA,1
130391,2,3,3


In [45]:
# Recode marital status
df_recode5 <- df_question5 %>%
  mutate(
    marital_status = ifelse(DMDMARTZ == 1, "Married", "Not Married")
  )
  df_recode5

SEQN,RIAGENDR,DPQ040,DMDMARTZ,marital_status
<dbl>,<dbl>,<dbl>,<dbl>,<chr>
130378,1,NA,1,Married
130379,1,0,1,Married
130380,2,1,1,Married
130386,1,0,1,Married
130387,2,0,3,Not Married
130388,2,NA,1,Married
130389,1,0,1,Married
130390,2,NA,1,Married
130391,2,3,3,Not Married


In [52]:
# Keep only rows where DPQ040 and marital_status are not missing
df_clean5 <- na.omit(df_recode5)
df_clean5

,SEQN,RIAGENDR,DPQ040,DMDMARTZ,marital_status
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2,130379,1,0,1,Married
3,130380,2,1,1,Married
4,130386,1,0,1,Married
5,130387,2,0,3,Not Married
7,130389,1,0,1,Married
9,130391,2,3,3,Not Married
10,130392,2,1,1,Married
11,130393,1,1,1,Married
12,130394,1,0,1,Married


In [53]:
# Keep only women in our dataframe (women =2)
df_women <- df_clean5[df_clean5$RIAGENDR == 2, ]
df_women

,SEQN,RIAGENDR,DPQ040,DMDMARTZ,marital_status
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
3,130380,2,1,1,Married
5,130387,2,0,3,Not Married
9,130391,2,3,3,Not Married
10,130392,2,1,1,Married
13,130395,2,3,3,Not Married
14,130396,2,1,1,Married
15,130397,2,1,1,Married
21,130404,2,0,2,Not Married
22,130406,2,0,1,Married


In [54]:
# Show number of married vs unmarried women
table(df_women$marital_status)


    Married Not Married 
       1455        1418 

In [55]:
# Create two groups for t-test
married_women_score<- subset(df_women, marital_status == "Married")$DPQ040
not_married_score <- subset(df_women, marital_status == "Not Married")$DPQ040

In [56]:
# Mean DPQ040 scores for both groups
mean(married_women_score, na.rm = TRUE)
mean(not_married_score, na.rm = TRUE)

[1] 0.8591065

[1] 1.065585

In [57]:
# Complete t-test
t_test_result <- t.test(married_women_score, not_married_score, var.equal = FALSE)
print(t_test_result)


	Welch Two Sample t-test

data:  married_women_score and not_married_score
t = -5.7919, df = 2840.4, p-value = 7.724e-09
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.2763807 -0.1365769
sample estimates:
mean of x mean of y 
0.8591065 1.0655853 



An independent samples t-test showed a significant difference between the two groups (t = –5.79, p < 0.001).
Married women reported lower tiredness compared to unmarried women in this sample.
